In [4]:
import os
import pandas as pd
path_to_incubations = "../../lab_work/sessions/incubations"

for file_name in os.listdir(path_to_incubations):
    path_to_file = os.path.join(path_to_incubations, file_name)
    print(path_to_file)
    df = pd.read_csv(path_to_file)
    print(df.columns)
    # print(path_to_file)
    

    


../../lab_work/sessions/incubations/2mLWetSpirullina.csv
Index(['sample_id', 'ratio',
       'amount to add  (in mg) to get 50 mg of C (or 100 mg of biomass)',
       'weight difference (from empty) in mg', 'notes',
       'incubation_start_date'],
      dtype='object')
../../lab_work/sessions/incubations/40mLDrySpirullina.csv
Index(['sample_id', 'ratio', 'biomass (mg)', 'salt (mg)', 'total weight (mg)',
       '5% Water (mg)', 'innoculum added', 'incubation_start_date'],
      dtype='object')
../../lab_work/sessions/incubations/2mLDrySpirullina.csv
Index(['sample_id', 'ratio',
       'amount to add  (in mg) to get 50 mg of C (or 100 mg of biomass)',
       'amount added (mg)', 'notes', 'incubation_start_date'],
      dtype='object')
../../lab_work/sessions/incubations/40mLWetSpirullina.csv
Index(['sample_id', 'ratio', 'biomass (mg)', 'salt (mg)', 'total weight (mg)',
       '170% Water (mg)', 'innoculum added', 'incubation_start_date'],
      dtype='object')
